# Questões 3 e 4

In [27]:
#Módulos utilizados:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [28]:
path = "teste_smarkio_lbs.xls"
df1 = pd.read_excel(path, 'Análise_ML')

In [29]:
df1.head()

,Pred_class,probabilidade,status,True_class
0,2,0.079892,approved,0.0
1,2,0.379377,approved,74.0
2,2,0.379377,approved,74.0
3,2,0.420930,approved,74.0
4,2,0.607437,approved,NaN


In [30]:
#Separar o que é approved para amostra de treino e validacao

df_ap = df1[df1['status']=='approved']

In [31]:
#Criação da variavel alvo: 1 para previu certo e 0 para previsoes erradas
df_ap['Classe_verdadeira'] = df_ap['True_class']
df_ap['Classe_verdadeira'] = np.where(df_ap['Classe_verdadeira'].notnull(),0,1)

In [32]:
df_ap.head()

,Pred_class,probabilidade,status,True_class,Classe_verdadeira
0,2,0.079892,approved,0.0,0
1,2,0.379377,approved,74.0,0
2,2,0.379377,approved,74.0,0
3,2,0.420930,approved,74.0,0
4,2,0.607437,approved,NaN,1


In [33]:
#Distribuição da variavel alvo
df_ap['Classe_verdadeira'].value_counts()

1    419
0    181
Name: Classe_verdadeira, dtype: int64

In [34]:
#Remocao das colunas status e True_class
df_ap.drop(columns=['status','True_class'],axis='columns',inplace=True)

In [35]:
df_ap.head()

,Pred_class,probabilidade,Classe_verdadeira
0,2,0.079892,0
1,2,0.379377,0
2,2,0.379377,0
3,2,0.420930,0
4,2,0.607437,1


In [36]:
df_ap.isnull().any()

Pred_class           False
probabilidade        False
Classe_verdadeira    False
dtype: bool

In [37]:
#Separacao da variavel dependente das variaveis explanatorias
X=df_ap.iloc[:,:-1]
y=df_ap.iloc[:,-1:]

## Treino de um Random Forest

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

modelo_rf = RandomForestClassifier(n_estimators=200,criterion = 'entropy', random_state=42)

#treino com cross validacao
scores_rf = cross_validate(modelo_rf,X,y,scoring=('accuracy','precision','recall','f1') )

scores_rf

{'fit_time': array([0.42928195, 0.39597821, 0.4306047 , 0.41760659, 0.41905832]),
 'score_time': array([0.02988601, 0.02393436, 0.02293825, 0.02503943, 0.02293825]),
 'test_accuracy': array([0.70833333, 0.71666667, 0.83333333, 0.76666667, 0.75      ]),
 'test_f1': array([0.78527607, 0.78481013, 0.88095238, 0.83529412, 0.84210526]),
 'test_precision': array([0.8       , 0.83783784, 0.88095238, 0.8255814 , 0.75471698]),
 'test_recall': array([0.77108434, 0.73809524, 0.88095238, 0.8452381 , 0.95238095])}

In [39]:
acuracia_media_rf = scores_rf['test_accuracy'].mean()
precisao_media_rf = scores_rf['test_precision'].mean()
recall_media_rf =  scores_rf['test_recall'].mean()
f1_media_rf = scores_rf['test_f1'].mean()
print('Médias: \n - acuracia:{},\n - precisao:{},\n - recall:{},\n - F1:{}'.
      format(acuracia_media_rf,precisao_media_rf,recall_media_rf,f1_media_rf))

Médias: 
 - acuracia:0.755,
 - precisao:0.8198177190542264,
 - recall:0.8375502008032129,
 - F1:0.825687592391849


## Treino de um XGBoost

In [40]:
from xgboost import XGBClassifier

modelo_x = XGBClassifier(n_estimators = 1000, learning_rate = 0.05, seed=42)

#treino com cross validacao
scores_x= cross_validate(modelo_x,X,y,scoring=('accuracy','precision','recall','f1') )


In [41]:
acuracia_media_x = scores_x['test_accuracy'].mean()
precisao_media_x = scores_x['test_precision'].mean()
recall_media_x =  scores_x['test_recall'].mean()
f1_media_x = scores_x['test_f1'].mean()
print('Médias: \n - acuracia:{},\n - precisao:{},\n - recall:{},\n - F1:{}'.
      format(acuracia_media_x,precisao_media_x,recall_media_x,f1_media_x))

Médias: 
 - acuracia:0.7466666666666667,
 - precisao:0.8196536553737207,
 - recall:0.8187894434882386,
 - F1:0.8175796902122275


## Métricas

Considerando:
- VP = verdadeiros positivos
- FP = falsos posistivos
- FN = falsos negativos
- VF = verdadeiros falsos

Vamos considerar as seguintes métricas:

- A Acurácia, que mede o numero de predições corretas em relação ao total de predições (VP+VF/VP+FP+FN+VF)
- A Precisão, que quantifica o número de verdadeiros positivos dentre todas as previsões positivas (VP/VP+FP)
- O Recall, que quantifica o numero de verdadeiros positivos dentro todos os exemplos positivos 
do dataset (VP/VP+FN)

## Previsão

Uma vez que as métricas do Random Forest foram ligeiramente superiores, vamos utiliza-lo para a previsão no set de dados com status de revision

In [42]:
#Filtrando do set de dados o status de revision
df_rev = df1[df1['status']=='revision']

In [43]:
#tamanho do set de dados
len(df_rev)

43

In [44]:
#Remocao das colunas status e True_class
df_rev.drop(columns=['status','True_class'],axis='columns',inplace=True)

In [45]:
modelo_escolhido = modelo_rf.fit(X,y)

In [46]:
pred_rf = modelo_escolhido.predict(df_rev)

In [47]:
df_rev['Predicao correta'] = pred_rf

In [48]:
#1 - previu certo e 0 - previu errado
df_rev

,Pred_class,probabilidade,Predicao correta
83,2,0.752448,1
164,3,0.784920,1
191,12,0.574756,0
201,17,0.506654,0
236,24,0.817525,1
237,24,0.909148,1
238,24,0.737133,1
239,24,0.318306,1
242,25,0.509871,1
243,25,0.629700,1


In [49]:
df_rev['Predicao correta'].value_counts()

1    33
0    10
Name: Predicao correta, dtype: int64